# Finger Joint Comparison

This notebook compares results from the IPL joint space width (JSW) analysis workflow to the results from the ORMIR_XCT package. To provide a more direct comparison, the same joint segmentation was used as input to the JSW analysis (generated using IPL). ORMIR_XCT results are shown with all combonations of oversampling and skeletonization.

## IPL JSW Results

The following distance transform image was generated in IPL:
<img src="images/IPL_JSW_DT_IMAGE.png" width="300" height="200">

The following JSW results were obtained from the above distance transform image:

| JSW Parameter | Value |
|:--------:|:--------:|
|  JS Volume (mm^3)  |  78.2068   |
|  Mean JS Width (mm)  |  1.8472   |
|  JS Max (mm)   |  2.6101   |
|  JS Min (mm)   |  1.3354   |
|  JS Asymmetry (JS Max / JS Min)   |  1.9545   |

# Step 1: Imports

Import modules/packages and set the input image path. 

In [1]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk

from ormir_xct.joint_space_analysis.jsw_morphometry import jsw_pad, jsw_dilate, jsw_erode, jsw_parameters

In [2]:
joint_seg_path = os.path.join("images", "IPL_vs_ORMIR_XCT_JOINT.nii")
output_path = "images"

# Step 2: Run the JSW Workflow

1. **Padding:** Pad the binary joint image with black space (zeros) to ensure the outside space is greater than the inside (joint) space.
2. **Dilation:** Next we dilate the binary image by a fixed constant (see `jsw_morphometry.py`) that is taken from the original IPL script. Here we use the SimpleITK Ball structural element for dilation which should be similar to the Euclidean metric used in IPL. Once dilation is complete, remove any islands and fill any holes in the dilated mask.
3. **Erosion:** Next erode the image and set the image's value to 30. Then, add the erosion and dilation images together. Areas with overlap between images will have a value of 90, and the joint space will have a value of 30. We can then use a binary threshold to extract the joint space. The joint space mask is then dilated to ensure correct distances are measured at the edges of the joint space.
4. **JSW Parameters:** Now we will compute JS volume, JSW mean, standard deviation, minimum, maximum, and JS asymmetry (JSW.Max / JSW.Min). Results are output to a CSV file.

In [3]:
filename = os.path.basename(joint_seg_path)
basename = os.path.splitext(filename)[0]
img = sitk.ReadImage(joint_seg_path, sitk.sitkUInt8)

# 1. Padding
print("Padding image...")
pad_image = jsw_pad(img)

# 2. Dilation
print("Dilating image...")
dilated_image = jsw_dilate(pad_image)
sitk.WriteImage(dilated_image, os.path.join(output_path, str(basename) + "_DILATE.nii"))

# 3. Erosion
print("Eroding image...")
eroded_image, js_mask, dilated_js_mask = jsw_erode(dilated_image, pad_image)
sitk.WriteImage(eroded_image, os.path.join(output_path, str(basename) + "_ERODE.nii"))
sitk.WriteImage(js_mask, os.path.join(output_path, str(basename) + "_JS_MASK.nii"))
sitk.WriteImage(dilated_js_mask, os.path.join(output_path, str(basename) + "_DILATED_JS_MASK.nii"))

Padding image...
Dilating image...
Eroding image...


# Step 3: Compute the JSW Parameters

All combinations of oversampling and skeletonization will be run through the distance transform algorithm to see which set of parameters best matches results from IPL.

In [4]:
# 4. JSW Parameters

# Oversampling=False, Skeletonization=False
dt_img1, jsw_params1 = jsw_parameters(pad_image, dilated_js_mask, basename,
                                      output_path, js_mask, pad_image.GetSpacing()[0], 
                                     oversamp=False, skel=False)
sitk.WriteImage(dt_img1, os.path.join(output_path, str(basename) + "_DT1.nii"))

# Oversampling=True, Skeletonization=False
dt_img2, jsw_params2 = jsw_parameters(pad_image, dilated_js_mask, basename,
                                      output_path, js_mask, pad_image.GetSpacing()[0], 
                                     oversamp=True, skel=False)
sitk.WriteImage(dt_img1, os.path.join(output_path, str(basename) + "_DT2.nii"))

# Oversampling=False, Skeletonization=True
dt_img3, jsw_params3 = jsw_parameters(pad_image, dilated_js_mask, basename,
                                      output_path, js_mask, pad_image.GetSpacing()[0], 
                                     oversamp=False, skel=True)
sitk.WriteImage(dt_img1, os.path.join(output_path, str(basename) + "_DT3.nii"))

# Oversampling=True, Skeletonization=True
dt_img4, jsw_params4 = jsw_parameters(pad_image, dilated_js_mask, basename,
                                      output_path, js_mask, pad_image.GetSpacing()[0], 
                                     oversamp=True, skel=True)
sitk.WriteImage(dt_img1, os.path.join(output_path, str(basename) + "_DT4.nii"))

# Step 4: Display Results

Use Pandas to print a table of the results we generated.

In [11]:
# Get the JS Volume
shape_stats = sitk.LabelShapeStatisticsImageFilter()
shape_stats.ComputeOrientedBoundingBoxOn()
shape_stats.Execute(js_mask)

stats_list = [
    (
        shape_stats.GetPhysicalSize(i),
        shape_stats.GetNumberOfPixels(i),
    )
    for i in shape_stats.GetLabels()
]

jsv = stats_list[0][0]

# Oversampling=False, Skeletonization=False
js_mean1 = jsw_params1[0][3]
js_max1 = jsw_params1[0][7]
js_min1 = jsw_params1[0][5]
js_as1 = js_max1 / js_min1

# Oversampling=True, Skeletonization=False
js_mean2 = jsw_params2[0][3]
js_max2 = jsw_params2[0][7]
js_min2 = jsw_params2[0][5]
js_as2 = js_max2 / js_min2

# Oversampling=False, Skeletonization=True
js_mean3 = jsw_params3[0][3]
js_max3 = jsw_params3[0][7]
js_min3 = jsw_params3[0][5]
js_as3 = js_max3 / js_min3

# Oversampling=True, Skeletonization=True
js_mean4 = jsw_params4[0][3]
js_max4 = jsw_params4[0][7]
js_min4 = jsw_params4[0][5]
js_as4 = js_max4 / js_min4


# Display a table with the IPL and ORMIR_XCT calculated JS parameters
data = {"ipl_results":  [78.2068, 1.8472, 2.6101, 1.3354, 1.9545],
        "oversamp=False, skel=False": [jsv, js_mean1, js_max1, js_min1, js_as1],
        "oversamp=True, skel=False": [jsv, js_mean1, js_max2, js_min2, js_as2],
        "oversamp=False, skel=True": [jsv, js_mean2, js_max3, js_min3, js_as3],
        "oversamp=True, skel=True": [jsv, js_mean3, js_max4, js_min4, js_as4],
        }
df = pd.DataFrame(data, index=["JS Volume (mm^3)", "Mean JS Width (mm)", "JS Max (mm)", 
                               "JS Min (mm)", "JS Asymmetry (JS Max / JS Min)"])

# Set table default options
pd.set_option('display.max_colwidth', None)
pd.set_option('colheader_justify', 'center')

df.style.format(precision=2)

,ipl_results,"oversamp=False, skel=False","oversamp=True, skel=False","oversamp=False, skel=True","oversamp=True, skel=True"
JS Volume (mm^3),78.21,80.47,80.47,80.47,80.47
Mean JS Width (mm),1.85,1.92,1.92,1.95,1.73
JS Max (mm),2.61,2.44,2.35,2.41,2.31
JS Min (mm),1.34,1.32,1.35,0.24,0.19
JS Asymmetry (JS Max / JS Min),1.95,1.85,1.75,9.94,12.05
